# Proyecto C

Estudiantes: 

1. Mariana Lozano Roncancio 202122878

# A. Presentaccion de modelo 

A continuacion se presenta el modlo con las siguientes cambios realizados:

- Adición de restricciones de continuidad y consumo de combustible.
- Inclusión de variables auxiliares para orden y combustible restante.
- Restricciones MTZ para evitar subtours.
- Inclusión de la restricción de visita única a cada cliente.
- Reorganización del modelo para mejorar su claridad y consistencia.

## 1. Conjuntos

- **P**: Conjunto de puntos de acceso (puertos)
- **D**: Conjunto de destinos (centros de consumo)
- **E**: Conjunto de estaciones de recarga
- **V**: Conjunto de vehículos
- **N**: Conjunto total de nodos, `N = P ∪ D ∪ E`
- **A**: Conjunto de arcos posibles, `A = {(i, j) ∈ N × N | i ≠ j}`

# 2. Parámetros
- `d_ij`: Distancia entre nodos i y j
- `C_ij`: Costo base entre nodos i y j
- `M_v`: Capacidad útil del vehículo v
- `A_v`: Autonomía del vehículo v
- `R_v`: Rendimiento (km/gal) del vehículo v
- `L_j`: Límite de peso en el nodo j
- `P_s`: Precio de galón en estación s
- `F_t`: Tarifa de flete (COP/km)
- `C_m`: Costo de mantenimiento (COP/km)
- `E_v`: Penalización por CO₂ por km para el vehículo v
- `T_ij`: Peaje base en tramo i → j
- `T2_ij`: Peaje adicional por tonelada
- `demand_j`: Demanda en el nodo j


## 3. Variables de decisión
- `x_ijv ∈ {0,1}`: 1 si el vehículo v viaja de i a j
- `q_ijv ≥ 0`: Carga transportada entre i y j por v
- `z_ijv ≥ 0`: Variable auxiliar para `q * x`
- `f_ijv ≥ 0`: Combustible consumido entre i y j
- `r_sv ≥ 0`: Galones recargados en la estación s por v
- `y_sv ∈ {0,1}`: 1 si v recarga en estación s
- `u_iv ≥ 0`: Orden de visita del nodo i por v
- `comb_iv ≥ 0`: Combustible restante en i por v

## 4. Función Objetivo

Minimizar el costo total de operación:

$$
\min \sum_v \sum_{(i,j)} \left( 
C_{ij} \cdot x_{ijv} + 
F_t \cdot d_{ij} \cdot x_{ijv} + 
C_m \cdot d_{ij} \cdot x_{ijv} + 
T_{ij} \cdot x_{ijv} + 
T2_{ij} \cdot z_{ijv} + 
E_v \cdot d_{ij} \cdot x_{ijv} 
\right) + \sum_v \sum_s P_s \cdot r_{sv}
$$


##  Restricciones

### 1. Conservación de flujo

$$
\sum_{j \in N, j \ne n} x_{jnv} = \sum_{j \in N, j \ne n} x_{njv}
\quad \forall n \in N, \forall v \in V
$$



### 2. Capacidad del vehículo

$$
q_{ijv} \le M_v \cdot x_{ijv}
\quad \forall (i,j) \in A, \forall v \in V
$$



### 3. Límite de peso por municipio

$$
q_{ijv} \le L_j
\quad \forall (i,j) \in A, j \in D, \forall v \in V
$$



### 4. Consumo de combustible

$$
f_{ijv} = \frac{d_{ij}}{R_v} \cdot x_{ijv}
\quad \forall (i,j) \in A, \forall v \in V
$$



### 5. Continuidad del combustible

$$
comb_{jv} = comb_{iv} - f_{ijv} + r_{jv}
\quad \forall (i,j) \in A, \forall v \in V
$$



### 6. Recarga en estación activa

$$
r_{sv} \le M \cdot y_{sv}
\quad \forall s \in E, \forall v \in V
$$



### 7. Eliminación de subtours (MTZ)

$$
u_{iv} - u_{jv} + |D| \cdot x_{ijv} \le |D| - 1
\quad \forall i \ne j \in D, \forall v \in V
$$



### 8. Satisfacción de la demanda

$$
\sum_{i \in N, i \ne j} \sum_{v \in V} q_{ijv} \ge demand_j
\quad \forall j \in D
$$



### 9. Linealización de \( z = q \cdot x \)

$$
\begin{aligned}
z_{ijv} &\le q_{ijv} \\
z_{ijv} &\le M_v \cdot x_{ijv} \\
z_{ijv} &\ge q_{ijv} - M_v \cdot (1 - x_{ijv})
\end{aligned}
$$

# B. Presentacion de Notebook ejecutado

In [3]:
!apt-get install -y coinor-cbc
!pip install pyomo


"apt-get" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [7]:
import pandas as pd
import pyomo.environ as pyo
from itertools import product

# Cargar archivos
vehicles = pd.read_csv("Datos\Vehicles2.csv")
stations = pd.read_csv("Datos\stations2.csv")  # No se usa en Caso 1
depots = pd.read_csv("Datos\depots2.csv")
clients = pd.read_csv("Datos\clients2.csv")

# Conjuntos
V = list(vehicles["VehicleID"].astype(str))
E = list(stations["EstationID"].astype(str))
P = list(depots["DepotID"].astype(str))
D = list(clients["LocationID"].astype(str))
N = P + D + E
A = [(i, j) for i in N for j in N if i != j]

# Parámetros
M_v = dict(zip(V, vehicles["Capacity"]))
A_v = dict(zip(V, vehicles["Range"]))
R_v = {v: 4 for v in V}
L_j = {j: 99999 for j in D}  # Asignar límites por defecto
demand = dict(zip(clients["LocationID"].astype(str), clients["Demand"]))
P_s = dict(zip(stations["EstationID"].astype(str), stations["FuelCost"]))

# Costos
dist = {(i, j): 1 for (i, j) in A}
C_ij = {(i, j): 1000 for (i, j) in A}
T_ij = {(i, j): 2000 for (i, j) in A}
T2_ij = {(i, j): 50 for (i, j) in A}
F_t = 5000
C_m = 700
E_v = {v: 10 for v in V}

model = pyo.ConcreteModel()
model.N = pyo.Set(initialize=N)
model.V = pyo.Set(initialize=V)
model.A = pyo.Set(dimen=2, initialize=A)
model.E = pyo.Set(initialize=E)
model.D = pyo.Set(initialize=D)

model.x = pyo.Var(model.A, model.V, domain=pyo.Binary)
model.q = pyo.Var(model.A, model.V, domain=pyo.NonNegativeReals)
model.z = pyo.Var(model.A, model.V, domain=pyo.NonNegativeReals)
model.y = pyo.Var(model.E, model.V, domain=pyo.Binary)
model.r = pyo.Var(model.E, model.V, domain=pyo.NonNegativeReals)
model.f = pyo.Var(model.A, model.V, domain=pyo.NonNegativeReals)
model.comb = pyo.Var(model.N, model.V, domain=pyo.NonNegativeReals)
model.u = pyo.Var(model.D, model.V, domain=pyo.NonNegativeReals)

# Inicialización de combustible
for v in V:
    for p in P:
        model.comb[p, v].fix(A_v[v])
    for n in N:
        if n not in P:
            model.comb[n, v].setlb(0)

model.z_restr = pyo.ConstraintList()
for (i, j) in A:
    for v in V:
        model.z_restr.add(model.z[i, j, v] <= model.q[i, j, v])
        model.z_restr.add(model.z[i, j, v] <= M_v[v] * model.x[i, j, v])
        model.z_restr.add(model.z[i, j, v] >= model.q[i, j, v] - M_v[v] * (1 - model.x[i, j, v]))

model.obj = pyo.Objective(
    expr=sum(
        (C_ij[i, j] + F_t * dist[i, j] + C_m * dist[i, j] + T_ij[i, j] + E_v[v] * dist[i, j]) * model.x[i, j, v] +
        T2_ij[i, j] * model.z[i, j, v]
        for (i, j) in model.A for v in model.V
    ) + sum(P_s[s] * model.r[s, v] for s in model.E for v in model.V),
    sense=pyo.minimize
)

def flujo_conservacion(model, n, v):
    return sum(model.x[i, n, v] for i in N if i != n) == sum(model.x[n, j, v] for j in N if j != n)
model.flujo = pyo.Constraint(N, model.V, rule=flujo_conservacion)

def restr_capacidad(model, i, j, v):
    return model.q[i, j, v] <= M_v[v] * model.x[i, j, v]
model.capacidad = pyo.Constraint(model.A, model.V, rule=restr_capacidad)

def restr_peso(model, i, j, v):
    if j in L_j:
        return model.q[i, j, v] <= L_j[j]
    return pyo.Constraint.Skip
model.peso = pyo.Constraint(model.A, model.V, rule=restr_peso)

def consumo_combustible(model, i, j, v):
    return model.f[i, j, v] == (dist[i, j] / R_v[v]) * model.x[i, j, v]
model.consumo = pyo.Constraint(model.A, model.V, rule=consumo_combustible)

def continuidad_combustible(model, i, j, v):
    return model.comb[j, v] == model.comb[i, v] - model.f[i, j, v] + (model.r[j, v] if j in E else 0)
model.continuidad = pyo.Constraint(model.A, model.V, rule=continuidad_combustible)

def recarga_estacion(model, s, v):
    return model.r[s, v] <= 9999 * model.y[s, v]
model.recarga = pyo.Constraint(model.E, model.V, rule=recarga_estacion)

def subtour_eliminacion(model, i, j, v):
    if i != j and i in D and j in D:
        return model.u[i, v] - model.u[j, v] + len(D) * model.x[i, j, v] <= len(D) - 1
    return pyo.Constraint.Skip
model.subtours = pyo.Constraint(model.A, model.V, rule=subtour_eliminacion)

def satisfacer_demanda(model, j):
    return sum(model.q[i, j, v] for i in N if i != j for v in V) >= demand[j]
model.demanda = pyo.Constraint(model.D, rule=satisfacer_demanda)

solver = pyo.SolverFactory('glpk')
results = solver.solve(model, tee=True)

if results.solver.status == pyo.SolverStatus.ok and results.solver.termination_condition == pyo.TerminationCondition.optimal:
    print("¡Solución óptima encontrada!")
    print("Valor objetivo:", pyo.value(model.obj))
else:
    print("No se encontró solución óptima.")
    print("Estado:", results.solver.status)
    print("Condición:", results.solver.termination_condition)




GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write C:\Users\57300\AppData\Local\Temp\tmpq0y8yq9a.glpk.raw --wglp C:\Users\57300\AppData\Local\Temp\tmpd03mc0lr.glpk.glp
 --cpxlp C:\Users\57300\AppData\Local\Temp\tmpc4njckm3.pyomo.lp
Reading problem data from 'C:\Users\57300\AppData\Local\Temp\tmpc4njckm3.pyomo.lp'...
C:\Users\57300\AppData\Local\Temp\tmpc4njckm3.pyomo.lp:91397: warning: lower bound of variable 'x2' redefined
C:\Users\57300\AppData\Local\Temp\tmpc4njckm3.pyomo.lp:91397: warning: upper bound of variable 'x2' redefined
15359 rows, 4460 columns, 38690 non-zeros
1110 integer variables, all of which are binary
92507 lines were read
Writing problem data to 'C:\Users\57300\AppData\Local\Temp\tmpd03mc0lr.glpk.glp'...
77346 lines were written
GLPK Integer Optimizer 5.0
15359 rows, 4460 columns, 38690 non-zeros
1110 integer variables, all of which are binary
Preprocessing...
60 constraint coefficient(s) were reduced
14379 rows, 4460 columns, 37710 